## import necessary packages 

In [1]:
from datetime import datetime
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## read in CSVs 

In [2]:
calls_derecho = pd.read_csv('./data/911_Phone_Calls_Derecho_050320_050420.csv')
calls_tornado = pd.read_csv('./data/911_Phone_Calls_Tornado_030320.csv')
dispatch_derecho = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)
dispatch_tornado = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
zipcodes = gpd.read_file('./data/zipcodes.geojson')
dispatch = pd.read_csv('./data/Metro_Nashville_Police_Department_Calls_for_Service.csv')

In [3]:
print(dispatch['Call Received'].dtype)

object


#metro_calls['Call Received'] = pd.to_datetime(metro_calls['Call Received'])

In [ ]:
dispatch[["call_date", "call_time"]]= dispatch.pop("Call Received").str.split(" ", n = 1, expand = True)


dispatch=dispatch[dispatch[‘call_date’].isin([‘03/03/2020’, ‘05/03/2020’, ‘05/04/2020’])]
dispatch=dispatch.reset_index(drop = True)
dispatch.head()

In [6]:
dispatch.head()

,Event Number,Complaint Number,Shift,Tencode,Tencode Description,Tencode Suffix,Tencode Suffix Description,Disposition Code,Disposition Description,Block,Street Name,Unit Dispatched,Sector,Zone,RPA,Latitude,Longitude,Mapped Location,call_date,call_time
0,PD202000711682,NaN,NaN,87,SAFETY HAZARD,P,PROGRESS,4,NaN,NaN,NaN,5250,H,513,9019.0,NaN,NaN,NaN,09/23/2020,02:26:09 PM
1,PD202000712516,NaN,NaN,83,SHOTS FIRED,P,PROGRESS,11,NaN,NaN,NaN,825B,N,617,4325.0,NaN,NaN,NaN,09/23/2020,09:47:43 PM
2,PD202000710687,NaN,NaN,93,TRAFFIC VIOLATION,NaN,NaN,3,NaN,NaN,NaN,5191,W,111N,5202.0,NaN,NaN,NaN,09/23/2020,05:28:57 AM
3,PD202000711464,NaN,NaN,96,BUSINESS CHECK,NaN,NaN,12,NaN,NaN,NaN,4E75,C,421,4095.0,NaN,NaN,NaN,09/23/2020,12:52:01 PM
4,PD201601010109,NaN,NaN,93,NaN,NaN,NaN,9T,NaN,NaN,NaN,321B,S,321,8706.0,NaN,NaN,NaN,10/03/2016,09:21:27 PM


In [ ]:
dispatch.shape

In [ ]:
dispatch.isnull().sum()

## Rename columns 

In [ ]:
calls_derecho = calls_derecho.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [ ]:
calls_tornado = calls_tornado.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [ ]:
dispatch_derecho = dispatch_derecho.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

In [ ]:
dispatch_tornado = dispatch_tornado.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

## Add geometry points to data sets 

In [ ]:
calls_derecho['Geometry'] = calls_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [ ]:
calls_tornado['Geometry'] = calls_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [ ]:
dispatch_derecho['Geometry'] = dispatch_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [ ]:
dispatch_tornado['Geometry'] = dispatch_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

## check data

In [ ]:
calls_derecho.head()

In [ ]:
calls_tornado.head()

In [ ]:
dispatch_derecho.head()

In [ ]:
dispatch_tornado.head()

In [ ]:
zipcodes.head()

## shape check

In [ ]:
calls_derecho.shape

In [ ]:
calls_tornado.shape

In [ ]:
dispatch_derecho.shape

In [ ]:
dispatch_tornado.shape

## count Cell Towers and Incident Descriptions

In [ ]:
calls_derecho['Cell Tower'].value_counts()

In [ ]:
calls_tornado['Cell Tower'].value_counts()

In [ ]:
dispatch_derecho['Incident Description'].value_counts()

In [ ]:
dispatch_tornado['Incident Description'].value_counts()

## count null values

In [ ]:
calls_derecho.isnull().sum()

In [ ]:
calls_tornado.isnull().sum()

In [ ]:
dispatch_derecho.isnull().sum()

In [ ]:
dispatch_tornado.isnull().sum()

map1 = sns.FacetGrid(dispactch_derecho, col = 'Incident Description')
map1.map(sns.barplot, "DateTime")

plt.hist(dispatch_derecho['Incident Description'], bins = 10);

## clean zipcodes data and create geodata frames for all datasets

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [ ]:
derecho_calls_geo = gpd.GeoDataFrame(calls_derecho, 
                           crs = zipcodes.crs, 
                           geometry = calls_derecho['Geometry'])
type(derecho_calls_geo)

In [ ]:
tornado_calls_geo = gpd.GeoDataFrame(calls_tornado, 
                           crs = zipcodes.crs, 
                           geometry = calls_tornado['Geometry'])
type(tornado_calls_geo)

In [ ]:
dispatch_derecho_geo = gpd.GeoDataFrame(dispatch_derecho, 
                           crs = zipcodes.crs, 
                           geometry = dispatch_derecho['Geometry'])
type(dispatch_derecho_geo)

In [ ]:
dispatch_tornado_geo = gpd.GeoDataFrame(dispatch_tornado, 
                           crs = zipcodes.crs, 
                           geometry = dispatch_tornado['Geometry'])
type(dispatch_tornado_geo)

## merge data sets with zipcodes

In [ ]:
derecho_calls_zip = gpd.sjoin(derecho_calls_geo, zipcodes, op = 'within')
derecho_calls_zip

In [ ]:
tornado_calls_zip = gpd.sjoin(tornado_calls_geo, zipcodes, op = 'within')
tornado_calls_zip

In [ ]:
dispatch_derecho_zip = gpd.sjoin(dispatch_derecho_geo, zipcodes, op = 'within')
dispatch_derecho_zip

In [ ]:
dispatch_tornado_zip = gpd.sjoin(dispatch_tornado_geo, zipcodes, op = 'within')
dispatch_tornado_zip

## nashville map 

In [ ]:
nashville = (36.16784, -86.77816)

nash_map = folium.Map(location = nashville, zoom_start = 10)

In [ ]:
poly_zip=zipcodes[zipcodes['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
poly_zip.shape

In [ ]:
tcalls_in_nw = tornado_calls_zip[tornado_calls_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
tcalls_in_nw.shape

In [ ]:
cluster_map = MarkerCluster().add_to(nash_map)

folium.GeoJson(poly_zip).add_to(nash_map)
marker_cluster = MarkerCluster().add_to(nash_map)
for row_index, row_values in tcalls_in_nw.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Cell Tower'])
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(cluster_map)

    
#cluster_map_nw.save('../maps/clusternw.html')

#cluster_map_nw


In [ ]:
nash_map

In [ ]:
#poly = zipcodes.loc[zipcodes['zip'] == '37207']
#polygon37207.shape